In [ ]:
from quchem_ibm.Qiskit_Chemistry import *

In [ ]:
transformation='BK'

In [ ]:
from quchem.Hamiltonian_Generator_Functions import *
from quchem.Graph import *
## HAMILTONIAN start

Molecule = 'LiH'
geometry = [('Li', (0., 0., 0.)), ('H', (0., 0., 1.45))]
basis = 'sto-6g'


### Get Hamiltonian
Hamilt = Hamiltonian_PySCF(Molecule,
                     run_scf=1, run_mp2=1, run_cisd=1, run_ccsd=1, run_fci=1,
                     basis=basis,
                     multiplicity=1,
                     geometry=geometry)  # normally None!
QubitHamiltonian = Hamilt.Get_Qubit_Hamiltonian(threshold=None, transformation=transformation)
### HAMILTONIAN end

#####################################

print(QubitHamiltonian)


fci_energy = Hamilt.molecule.fci_energy
print(fci_energy)

From PHYS. REV. X, **8**, 031022 (2018):

$$LiH_{BK}^{HartreeFock} = | 101000000000\rangle$$

In [ ]:
from quchem.Ansatz_Generator_Functions import *
n_electrons=Hamilt.molecule.n_electrons
n_qubits=Hamilt.molecule.n_qubits

ansatz_obj = Ansatz(n_electrons,n_qubits)
print('JW ground state = ', ansatz_obj.Get_JW_HF_state_in_OCC_basis())
print('BK ground state = ', ansatz_obj.Get_BK_HF_state_in_OCC_basis())

In [ ]:
# qubits_to_remove = Find_I_Z_indices_in_Hamiltonian(QubitHamiltonian, Hamilt.molecule.n_qubits)
# print('qubits only acted on by I or Z:', qubits_to_remove)


# input_state = ansatz_obj.Get_BK_HF_state_in_OCC_basis() if transformation=='BK' else ansatz_obj.Get_JW_HF_state_in_OCC_basis()
# # Remove_Z_terms_from_Hamiltonian
# NewQubitHamiltonian = Remove_Z_terms_from_Hamiltonian(
#                                                     QubitHamiltonian, 
#                                                     input_state,
#                                                     qubits_to_remove,
#                                                     check_reduction=True)
# NewQubitHamiltonian

In [ ]:
# qubitNo_re_label_dict, NewQubitHamiltonian_relabelled = Re_label_Hamiltonian(NewQubitHamiltonian)
# NewQubitHamiltonian_relabelled

In [ ]:
NewQubitHamiltonian_relabelled=QubitHamiltonian

# Find what new FCI energy is
- here should be the same as true answer!

In [ ]:
from openfermion import qubit_operator_sparse
from scipy.sparse import csc_matrix
new_Molecular_H_MATRIX =  csc_matrix(qubit_operator_sparse(NewQubitHamiltonian_relabelled))

# new_Molecular_H_MATRIX = np.flip(new_Molecular_H_MATRIX)

from scipy.sparse.linalg import eigs
try:
    eig_values, eig_vectors = eigs(new_Molecular_H_MATRIX)
except:
    from scipy.linalg import eig
    eig_values, eig_vectors = eig(new_Molecular_H_MATRIX.todense())
    
new_FCI_Energy = min(eig_values)


index = np.where(eig_values==new_FCI_Energy)[0][0]
ground_state_vector = eig_vectors[:, index]

print('new_FCI = ', new_FCI_Energy, 'VS old FCI:', fci_energy)
print(np.isclose(new_FCI_Energy, fci_energy))

In [ ]:
np.dot(ground_state_vector.conj().T, new_Molecular_H_MATRIX.dot(ground_state_vector))

# HF + UCCSD

In [ ]:
# input_state = ansatz_obj.Get_BK_HF_state_in_OCC_basis() if transformation=='BK' else ansatz_obj.Get_JW_HF_state_in_OCC_basis()
# new_input_state = np.delete(input_state, qubits_to_remove)

# n_qubits_new = len(new_input_state)
# new_input_state

In [ ]:
new_input_state=ansatz_obj.Get_BK_HF_state_in_OCC_basis()

In [ ]:
# ansatz_obj.Get_ia_and_ijab_terms()

# print('ia standard')
# print(ansatz_obj.Sec_Quant_CC_ia_Fermi_ops)

# print('ijab standard')
# print(ansatz_obj.Sec_Quant_CC_ijab_Fermi_ops)

# ### single trotter step
# ansatz_obj.UCCSD_single_trotter_step(transformation,
#                                      List_FermiOps_ia=ansatz_obj.Sec_Quant_CC_ia_Fermi_ops,
#                                      List_FermiOps_ijab=ansatz_obj.Sec_Quant_CC_ijab_Fermi_ops)

# print('')
# print('ia standard')
# print(ansatz_obj.Second_Quant_CC_single_Trot_list_ia)
# print('ijab standard')
# print(ansatz_obj.Second_Quant_CC_single_Trot_list_ijab)

In [ ]:
# new_CC_ia_single_trot = Remove_indices_from_qubit_ops(ansatz_obj.Second_Quant_CC_single_Trot_list_ia,
#                                                       qubits_to_remove)
# print('ia reduced')
# print(new_CC_ia_single_trot)

# new_CC_ijab_single_trot = Remove_indices_from_qubit_ops(ansatz_obj.Second_Quant_CC_single_Trot_list_ijab,
#                                                       qubits_to_remove)
# print('ijab reduced')
# print(new_CC_ijab_single_trot)


# relabelled_new_CC_ia_single_trot =Re_label_qubit_operators(qubitNo_re_label_dict, new_CC_ia_single_trot)
# relabelled_new_CC_ijab_single_trot =Re_label_qubit_operators(qubitNo_re_label_dict, new_CC_ijab_single_trot)

# print('')
# print('ia reduced relabelled')
# print(relabelled_new_CC_ia_single_trot)
# print('')
# print('ijab reduced relabelled')
# print(relabelled_new_CC_ijab_single_trot)

# Ansatz Circuit

In [ ]:
# for op in relabelled_new_CC_ia_single_trot[1]:
#     print(op)
# print(list(op.terms.keys())[0])

In [ ]:
# UCCSD_ansatz_Q_Circ_obj = Ansatz_Circuit(new_input_state,
#                                      relabelled_new_CC_ia_single_trot, 
#                                      relabelled_new_CC_ijab_single_trot)

# theta_ia = [0 for _ in range(len(relabelled_new_CC_ia_single_trot))]
# theta_ijab = [0 for _ in range(len(relabelled_new_CC_ijab_single_trot))]

# UCCSD_ansatz_Q_Circ =UCCSD_ansatz_Q_Circ_obj.Get_Full_HF_UCCSD_QC(
#                                         Theta_param_list_ia=theta_ia, 
#                                          Theta_param_list_ijab=theta_ijab,
#                                          ia_first=True)
# UCCSD_ansatz_Q_Circ

In [ ]:
n_qubits= len(new_input_state)
check_ansatz_state = False
decompose_fully=True

Ansatz_circuit, q_reg = Vector_defined_Ansatz(n_qubits,
                                              ground_state_vector,
                                              check_ansatz_state=check_ansatz_state,
                                              decompose_fully=decompose_fully)

In [ ]:
Ansatz_circuit.draw()

# Standard VQE

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
standard_VQE_circuits, standard_I_term = Build_Standard_VQE_circuits_MEMORY_EFF(
                                                    NewQubitHamiltonian_relabelled, 
                                                    Ansatz_circuit,
                                                    q_reg)

In [ ]:
# standard_VQE_circuits, standard_I_term = Build_Standard_VQE_circuits(
#                                                     NewQubitHamiltonian_relabelled, 
#                                                     Ansatz_circuit,
#                                                     q_reg)

# Graph

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
Hamiltonian_graph_obj = Openfermion_Hamiltonian_Graph(NewQubitHamiltonian_relabelled)

commutativity_flag = 'AC' ## <- defines relationship between sets!!!
plot_graph = False
Graph_colouring_strategy='largest_first'
anti_commuting_sets = Hamiltonian_graph_obj.Get_Clique_Cover_as_QubitOp(commutativity_flag, Graph_colouring_strategy=Graph_colouring_strategy, plot_graph=plot_graph)

anti_commuting_sets

# Seq Rot circuits

In [ ]:
n_qubits= len(new_input_state)
rotation_reduction_check=False

Seq_Rot_VQE_circuits, Seq_Rot_I_term = Get_Seq_Rot_Unitary_Part_circuits_MEMORY_EFF(
                                anti_commuting_sets, 
                                Ansatz_circuit,
                                q_reg,
                                n_qubits,
                                S_index_dict=None, 
                                rotation_reduction_check=rotation_reduction_check)

In [ ]:
# n_qubits= len(new_input_state)
# rotation_reduction_check=False

# Seq_Rot_VQE_circuits, Seq_Rot_I_term = Get_Seq_Rot_Unitary_Part_circuits(
#                                 anti_commuting_sets, 
#                                 Ansatz_circuit,
#                                 q_reg,
#                                 n_qubits,
#                                 S_index_dict=None, 
#                                 rotation_reduction_check=rotation_reduction_check)

# LCU circuits

In [ ]:
# n_qubits= len(new_input_state)
# check_ansatz_state = False

# LCU_VQE_circuits, LCU_I_term= Get_LCU_Unitary_Part_circuits(anti_commuting_sets, ground_state_vector, n_qubits, 
#                                   N_index_dict=None,check_ansatz_state=check_ansatz_state)

In [ ]:
n_qubits= len(new_input_state)


LCU_VQE_circuits, LCU_I_term=Get_LCU_Unitary_Part_circuits_MEMORY_EFF(
                                                                        anti_commuting_sets, 
                                                                        Ansatz_circuit,
                                                                        q_reg,
                                                                        n_qubits,
                                                                        N_index_dict=None)

# Save experiment input

In [ ]:
list(NewQubitHamiltonian_relabelled)

In [ ]:
filename = 'LiH_bravyi_kitaev_12_qubit_experiment'
n_qubits= len(new_input_state)

Save_exp_inputs(filename, NewQubitHamiltonian_relabelled, anti_commuting_sets, Hamilt.geometry, basis, transformation,
                    Graph_colouring_strategy, fci_energy,
                    standard_VQE_circuits, standard_I_term,
                    Seq_Rot_VQE_circuits, Seq_Rot_I_term,
                    LCU_VQE_circuits, LCU_I_term,
                    ground_state_vector,
                    n_qubits,
                    S_index_dict=None,
                    N_index_dict=None)

In [ ]:
# qc_list = [QuantumCircuit.from_qasm_str(exp_dict['circuit']) for
#            exp_dict in Seq_Rot_VQE_circuits if exp_dict['circuit']]